In [1]:
import pandas as pd
import os
import numpy as np
from PIL import Image
import shutil
from pylab import array, arange, uint8
import cv2


Función retina_fundus es para el preprocesado de una imagen, dada una imagen de fondo de retina dada, recorta la imagen a un cuadrado en torno al círculo que contendría la información sobre la image retinal, y la reescala a 299px x 299px, devolviendo así una imagen de dichas dimensiones.

In [2]:
def retina_fundus(im):
    #
    diameter=518
    #
    
    
    copy = im.copy()
    
    contours= None
    #aumentar contraste
    imax = 255.0
    im=(imax/1.3)*(im/(imax/1.5))**2
    
    #escala de grises
    #gray = im.convert('L')
    #uint_img = np.array(im*255).astype('uint8')

    #grayImage = cv2.cvtColor(uint_img, cv2.COLOR_GRAY2BGR)
    
    gray = cv2.cvtColor(im.astype('uint8') * 255, cv2.COLOR_BGR2GRAY)
    #encontrar contornos
    cnts = cv2.findContours(gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[-2]
    center = None
    
    if len(cnts) > 0:
        # encuentra el contorno más grande
        c = max(cnts, key=cv2.contourArea)
        ((x, y), radius) = cv2.minEnclosingCircle(c)

        # tienen que tener un radio mínimo de 100 px
        if radius > 100:
            M = cv2.moments(c)
            center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))
            contours=(center,radius)
            
    #descartamos la imágenes que no cumplen el requisitoo mínimo
    if contours is None:
        return np.array([])
    
    
    center, radius = contours
        
     
    # Calculamos los limites de la imagen para recortarla
    x_min = max(0, int(center[0] - radius))
    y_min = max(0, int(center[1] - radius))
    z = int(radius*2)
    x_max = x_min + z
    y_max = y_min + z

    # Recortamos la imagen
    copy = copy[y_min:y_max, x_min:x_max]
   
    # Scale the image.
    fx = fy = (diameter / 2) / radius
    copy = cv2.resize(copy, (0, 0), fx=fx, fy=fy)
        
    # Add padding to image.
    shape = copy.shape

    # Get the border shape size.
    top = bottom = int((diameter - shape[0])/2)
    left = right = int((diameter - shape[1])/2)

    # Add 1 pixel if necessary.
    if shape[0] + top + bottom == diameter - 1:
        top += 1

    if shape[1] + left + right == diameter - 1:
        left += 1

    # Define border of the image.
    border = [top, bottom, left, right]

    # Add border.
    copy = cv2.copyMakeBorder(copy, *border,
                              borderType=cv2.BORDER_CONSTANT,
                              value=[0, 0, 0])
    # Return the image.
    return copy    
        
        
        
        
        
        

In [3]:
def preprocesado(csv, destino1 , destino2 , origen):
    Labels = pd.read_csv(csv)
    listing = os.listdir(origen)
    
    folders_to_be_created = np.unique(list(str(Labels['level'])))

    source = os.getcwd()

    for new_path in folders_to_be_created:
        if((new_path=='0')or(new_path=='1')or(new_path=='2')or(new_path=='3')or(new_path=='4')):
            if not os.path.exists(".//"+destino1+new_path):
                os.makedirs(destino1+new_path)
                
            if(new_path!='0'):
                new_path='1'
            if not os.path.exists(".//"+destino2+new_path):
                os.makedirs(destino2+new_path)
                
                
                
                
                
    n1=0

    for file in listing:
        if(n1 % 100 == 0):
            print("numero",n1)
            
        base = os.path.basename(origen + file)
        fileName = os.path.splitext(base)[0]
        
        label=Labels.loc[Labels.image==fileName, 'level'].values[0]
        label2=label
        if(label2>0):
            label2=1
        
        #im = Image.open(origen + file)  
        #print(" ",origen + file)
        im = cv2.imread(os.path.abspath(origen + file))
        im1= retina_fundus(im)

        if(im1.size != 0):
            
            im1 = cv2.cvtColor(im1, cv2.COLOR_BGR2RGB)
            res_img = Image.fromarray(im1)
            
            res_img.save(destino1+str(label)+"//"+file)
            
            res_img.save(destino2+str(label2)+"//"+file)
        
        
        
        
        n1=n1+1

In [4]:

preprocesado("trainLabels.csv","data_518//test1_5cat//","data_518//test1_2cat//","data//test1//")

numero 0
numero 100
numero 200
numero 300
numero 400
numero 500
numero 600
numero 700
numero 800
numero 900
numero 1000
numero 1100
numero 1200
numero 1300
numero 1400
numero 1500
numero 1600
numero 1700
numero 1800
numero 1900
numero 2000
numero 2100
numero 2200
numero 2300
numero 2400
numero 2500
numero 2600
numero 2700
numero 2800
numero 2900
numero 3000
numero 3100
numero 3200
numero 3300
numero 3400
numero 3500
numero 3600
numero 3700
numero 3800
numero 3900
numero 4000
numero 4100
numero 4200
numero 4300
numero 4400
numero 4500
numero 4600
numero 4700
numero 4800
numero 4900
numero 5000
numero 5100
numero 5200
numero 5300
numero 5400
numero 5500
numero 5600
numero 5700
numero 5800
numero 5900
numero 6000
numero 6100
numero 6200
numero 6300
numero 6400
numero 6500
numero 6600
numero 6700
numero 6800
numero 6900
numero 7000
numero 7100
numero 7200
numero 7300
numero 7400
numero 7500
numero 7600
numero 7700
numero 7800
numero 7900
numero 8000
numero 8100
numero 8200
numero 8300
nume

In [5]:

preprocesado("trainLabels.csv","data_518//val1_5cat//","data_518//val1_2cat//","data//val1//")

numero 0
numero 100
numero 200
numero 300
numero 400
numero 500
numero 600
numero 700
numero 800
numero 900
numero 1000
numero 1100
numero 1200
numero 1300
numero 1400
numero 1500
numero 1600
numero 1700
numero 1800
numero 1900
numero 2000
numero 2100
numero 2200
numero 2300
numero 2400
numero 2500
numero 2600
numero 2700
numero 2800
numero 2900
numero 3000
numero 3100
numero 3200
numero 3300
numero 3400
numero 3500
numero 3600
numero 3700
numero 3800
numero 3900
numero 4000
numero 4100
numero 4200
numero 4300
numero 4400
numero 4500
numero 4600
numero 4700
numero 4800
numero 4900
numero 5000
numero 5100
numero 5200
numero 5300
numero 5400
numero 5500
numero 5600
numero 5700
numero 5800
numero 5900
numero 6000
numero 6100
numero 6200
numero 6300
numero 6400
numero 6500
numero 6600
numero 6700
numero 6800
numero 6900
numero 7000
numero 7100
numero 7200
numero 7300
numero 7400
numero 7500
numero 7600
numero 7700
numero 7800
numero 7900
numero 8000
numero 8100
numero 8200
numero 8300
nume